# HW 1 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом отработать подходы оптимизации параметров нейросети (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/)

In [1]:
import random
import time
import torch
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

torch.manual_seed(0)

In [2]:
import numpy
print (numpy.__version__)

1.22.3


In [3]:
import tensorly as tl
from tensorly.decomposition import partial_tucker 
from tensorly import unfold
from tensorly import fold

In [4]:
def unfold(tensor, mode=0):
    """Unfolds a tensors following the Kolda and Bader definition

        Moves the `mode` axis to the beginning and reshapes in Fortran order
    """
    return torch.reshape(torch.moveaxis(tensor, mode, 0), 
                      (tensor.shape[mode], -1))
def fold(unfolded_tensor, mode, shape):
    """Refolds the mode-`mode` unfolding into a tensor of shape `shape`
    
    Parameters
    ----------
    unfolded_tensor : ndarray
        unfolded tensor of shape ``(shape[mode], -1)``
    mode : int
        the mode of the unfolding
    shape : tuple
        shape of the original tensor before unfolding
    
    Returns
    -------
    ndarray
        folded_tensor of shape `shape`
    """
    full_shape = list(shape)
    mode_dim = full_shape.pop(mode)
    full_shape.insert(0, mode_dim)
    return torch.moveaxis(torch.reshape(unfolded_tensor, full_shape), 0, mode)
def mode_dot(tensor, matrix_or_vector, mode):
    """n-mode product of a tensor by a matrix at the specified mode.

    Parameters
    ----------
    tensor : ndarray
        tensor of shape ``(i_1, ..., i_k, ..., i_N)``
    matrix_or_vector : ndarray
        1D or 2D array of shape ``(J, i_k)`` or ``(i_k, )``
        matrix or vectors to which to n-mode multiply the tensor
    mode : int

    Returns
    -------
    ndarray
        `mode`-mode product of `tensor` by `matrix_or_vector`
        * of shape :math:`(i_1, ..., i_{k-1}, J, i_{k+1}, ..., i_N)` if matrix_or_vector is a matrix
        * of shape :math:`(i_1, ..., i_{k-1}, i_{k+1}, ..., i_N)` if matrix_or_vector is a vector
    """
    # the mode along which to fold might decrease if we take product with a vector
    fold_mode = mode
    new_shape = list(tensor.shape)

    # tensor times vector case: make sure the sizes are correct 
    # (we are contracting over one dimension which then disappearas)
    if matrix_or_vector.ndim == 1: 
        if len(new_shape) > 1:
            new_shape.pop(mode)
            fold_mode -= 1
        else:
            new_shape = [1]
    # This is the actual operation: we use the equivalent formulation of the n-mode-product using the unfolding
    res = torch.matmul(matrix_or_vector, unfold(tensor, (mode))) 

    new_shape[mode] = matrix_or_vector.shape[0]
    return fold(res, mode, new_shape)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [7]:
from numpy.linalg import svd
from numpy.random import randint
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=10):
  # data - тензор с заданной размерностью
  # U - список матриц
  # G - ядро разложения
    G = torch.tensor(randint(0, 9, r**3).reshape(r,r,r)) 
    G = G.double()
    U = []
    data = G.detach().clone()
    for i in range(0, 3):
        A = torch.tensor(randint(0, 9, r*size[i]).reshape(size[i], r))
        A = A.to(dtype=torch.float64)
        data = mode_dot(data, A, mode= i)
        U.append(A)
    data = torch.floor(data*9/(data.max() - data.min()))
    data += torch.ones(100, 200, 150)*1e-2
    #data = data.to(dtype=torch.float64)
    return U, data, G

In [8]:
U,data,core = get_tensor(size=(400, 500, 500), r = 20)

In [9]:
data.shape

torch.Size([400, 500, 500])

Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

## 2 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [10]:
#!pip install tensorly
import tensorly as tl
from tensorly.decomposition import tucker

In [11]:
data.shape

torch.Size([400, 500, 500])

In [12]:
from tensorly.decomposition import tucker
X = tl.tensor(data)
core, factors = tucker(X, rank=[20, 20, 20])
core_torch = torch.from_numpy(core)
factors_torch = []
for factor in factors:
    factors_torch.append(torch.from_numpy(factor))

In [13]:
core.shape

(20, 20, 20)

In [14]:
Decomposed = core_torch.detach().clone()
i = 0
for factor in factors_torch:
    Decomposed = mode_dot(Decomposed, factor, i)
    i += 1


In [15]:
Decomposed.shape

torch.Size([400, 500, 500])

Не забудьте померить ошибку разложения по метрике MSE

In [16]:
import torch.nn.functional as F
loss = torch.norm(Decomposed.data - data.data, 2)/torch.norm(data.data, 2)
print(loss)

tensor(0.0137, dtype=torch.float64)


## 3 Реализуйте разложение градиентным методом

### 3.1 Реализуйте метод для восстановления тензора по разложению

In [17]:
def repair_tensor(G_, U):
  # data - восстановленный тензор из матриц и ядра
  # U - список матриц
  # G_ - ядро разложения
    data = G_.detach().clone()
    i = 0
    for u in U:
        data = mode_dot(data, u, i)
        data.retain_grad()
        i += 1
    return data

### 3.2 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.

In [34]:
import math
import torch
from torch.optim.optimizer import Optimizer
from torch import autograd


class Opt(Optimizer):
    def __init__(self, params, # params: model.parameters()
                       lr: float = 1e-3, # input: type = value
                 weight_decay=0.
                 ): 
        # constructor
        defaults = dict(lr=lr, weight_decay=weight_decay) 
        # add a default attribute that can be accessed
        super(Opt, self).__init__(params, defaults)

    def step(self, closure=None): 
        loss= None
        for group in self.param_groups:
            weight_decay = group['weight_decay']

            for param in group['params']:
                if param.grad is None:
                    continue
                grad_param = param.grad.data
                
                if weight_decay != 0:
                    grad_param += weight_decay*param.data

                param.data -= group['lr']*grad_paramcx 

        return loss

In [35]:
X = data

In [36]:
from tensorly import backend as T
from torch import nn
from mxnet import nd, autograd

### 3.3 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [103]:
%%time


def Decomp(r, X):
    
    torch.manual_seed(42)
    core = torch.randn(r,r,r, requires_grad=True, dtype=torch.float64)
    factors = [torch.rand(X.shape[i], r, requires_grad=True, dtype=torch.float64) for i in range(3)] 
    core.cuda()
    for i in factors:
        i.cuda()

    n_iter = 1201
    lr = 1e-4
    penalty = 0.6
    X.cuda()

    optimizer = Opt([core]+factors, lr)

    for i in range(1, n_iter):
        optimizer.zero_grad()


    # Reconstruct the tensor from the decomposed form
        rec = repair_tensor(core, factors)

    # squared l2 loss
        loss = torch.norm(rec - X, 2)

    # squared l2 penalty on the factors of the decomposition
        for f in factors:
            loss = loss + penalty * f.pow(2).sum()
        loss.backward()
        optimizer.step()

        rec_error = 0
        if i % 100 == 0:
            rec_error = torch.norm(rec.data - X.data, 2)/torch.norm(X.data, 2)
            print("Epoch {},. Rec. error: {}".format(i, rec_error))
    return rec_error


Wall time: 0 ns


In [104]:
import time

## 4 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [105]:
tensors = []
loss_paket = []
loss_my = []

time_exec_my = []
time_exec_pocket = []



for i in range(7):
    print('{}-tensor'.format(i))
    size = (randint(100, 300),randint(100, 300),randint(100, 300))
    t2nsor = get_tensor(size, r=10)[1]
    tensors.append(t2nsor)
    
    start_time = time.time()
    print(t2nsor.shape)
    The_loss = Decomp(10, t2nsor)
    time_exec_my.append(start_time - time.time())
    loss_my.append(The_loss)
    
    start_time = time.time()
    
    X = tl.tensor(t2nsor)
    core, factors = tucker(X, rank=[20, 20, 20])    
    
    core_torch = torch.from_numpy(core)
    factors_torch = []
    for factor in factors:
        factors_torch.append(torch.from_numpy(factor.copy()))
    
    Decomposed = core_torch.detach().clone()
    i = 0
    for factor in factors_torch:
        Decomposed = mode_dot(Decomposed, factor, i)
        i += 1
    loss_paket.append(torch.norm(Decomposed.data - t2nsor.data, 2)/torch.norm(t2nsor.data, 2))
    time_exec_pocket.append(time.time() - start_time)

0-tensor
torch.Size([190, 297, 145])
Epoch 100,. Rec. error: 0.5062390727699241
Epoch 200,. Rec. error: 0.30320337524080654
Epoch 300,. Rec. error: 0.22178615699745957
Epoch 400,. Rec. error: 0.1837834499285692
Epoch 500,. Rec. error: 0.1643184007553727
Epoch 600,. Rec. error: 0.15364186585230777
Epoch 700,. Rec. error: 0.14740561679374586
Epoch 800,. Rec. error: 0.14353424287483538
Epoch 900,. Rec. error: 0.14099326417843364
Epoch 1000,. Rec. error: 0.13924348516974147
Epoch 1100,. Rec. error: 0.1379894952938641
Epoch 1200,. Rec. error: 0.13706093934797892
1-tensor
torch.Size([248, 234, 217])
Epoch 100,. Rec. error: 0.4358419991908741
Epoch 200,. Rec. error: 0.2664039233100596
Epoch 300,. Rec. error: 0.19572607429741753
Epoch 400,. Rec. error: 0.16166162278546686
Epoch 500,. Rec. error: 0.14410567420491374
Epoch 600,. Rec. error: 0.13462554287001305
Epoch 700,. Rec. error: 0.12921075919910358
Epoch 800,. Rec. error: 0.12592084869182749
Epoch 900,. Rec. error: 0.12380414161296888
Epoch

In [108]:
for i in range(len(loss_my)):
    print(loss_my[i], loss_paket[i])

tensor(0.1371, dtype=torch.float64) tensor(0.0676, dtype=torch.float64)
tensor(0.1206, dtype=torch.float64) tensor(0.0613, dtype=torch.float64)
tensor(0.1743, dtype=torch.float64) tensor(0.0757, dtype=torch.float64)
tensor(0.1490, dtype=torch.float64) tensor(0.0686, dtype=torch.float64)
tensor(0.1552, dtype=torch.float64) tensor(0.0718, dtype=torch.float64)
tensor(0.1360, dtype=torch.float64) tensor(0.0672, dtype=torch.float64)
tensor(0.1420, dtype=torch.float64) tensor(0.0662, dtype=torch.float64)


In [110]:
print("mean error of my decomposition = ",np.array(loss_my).mean())
print("mean error of module decomposition = ",np.array(loss_paket).mean())

mean error of my decomposition =  0.1448935718294205
mean error of module decomposition =  0.06834384146763782


In [107]:
ranks = [5, 5, 5]

core = T.tensor(rng.random_sample(ranks))
factors = [T.tensor(rng.random_sample((tensor.shape[i], ranks[i]))) for i in range(tensor.ndim)]

NameError: name 'rng' is not defined

In [861]:
core = torch.randn(r, requires_grad=True)

In [863]:
print(core.grad)

None


In [864]:
core

tensor([-0.9806, -0.2274, -0.3693, -1.0661, -1.1803, -1.1742, -0.0673, -0.3317,
         3.0527, -0.7597, -0.8742, -1.5350,  0.6282, -0.5962,  0.4910, -0.0620,
         0.7298, -1.1375,  0.4975, -1.6329], requires_grad=True)

tensor([[[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.